<h1 align ="center"> Python SDK Sample</h1>
<hr>

# Chat Completions

Chat models take a series of messages as input, and return a model-generated message as output.
The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message). 

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

In [1]:
# import the OpenAI Python library for calling the OpenAI API
import openai

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f18ee8fa9b0>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


### Setup Parameters

In [3]:
keyvault = ws.get_default_keyvault()

aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")


In [4]:
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = aoai_key

In [19]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

In [33]:
base_system_message = "You are a helpful assistant that converts SAS code into Python."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant that converts SAS code into Python.


In [40]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = f"""data work.experience;
input employee $ experience;
datalines;
Ahmet 10
Fatma 8
Ali 12
Elif 7
John 4
Jade 15
;
run;
"""

In [41]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [42]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
# You can get "resource not found error in case your version isn't correct"

model_name= "gpt-4-32k"

try:
    max_response_tokens = 500

    response = send_message(messages, model_name, max_response_tokens)
    messages.append({"role": "assistant", "content": response})

    print_conversation(messages)    
   
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

[SYSTEM]
You are a helpful assistant that converts SAS code into Python.

[USER]
data work.experience;
input employee $ experience;
datalines;
Ahmet 10
Fatma 8
Ali 12
Elif 7
John 4
Jade 15
;
run;


[ASSISTANT]
{
  "id": "chatcmpl-7nZDeUoNnlDo0YijTaISrXBLHdElS",
  "object": "chat.completion",
  "created": 1692047810,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "In Python, the SAS code can be converted using pandas DataFrame as follows:\n\n```python\nimport pandas as pd\n\n# Define the data\ndata = {'employee': ['Ahmet', 'Fatma', 'Ali', 'Elif', 'John', 'Jade'],\n        'experience': [10, 8, 12, 7, 4, 15]}\n\n# Create DataFrame\nwork_experience = pd.DataFrame(data)\n\nprint(work_experience)\n```\n\nThis code will create a DataFrame `work_experience` with the same data as in your SAS dataset. The `print` statement will display the DataFrame."
      }
    }
  ],
  "usage": {

**Second Example**

In [ ]:
messages.clear()

In [58]:
base_system_message = "You are a helpful assistant that converts natural language into SQL."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant that converts natural language into SQL.


In [59]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = f"""Write sql code whichmerges 2 table and finds the maximum value of salary column"""

In [61]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [62]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
# You can get "resource not found error in case your version isn't correct"

model_name= "gpt-4-32k"

try:
    max_response_tokens = 500

    response = send_message(messages, model_name, max_response_tokens)
    messages.append({"role": "assistant", "content": response})

    print_conversation(messages)    
   
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

[SYSTEM]
You are a helpful assistant that converts natural language into SQL.

[USER]
Write sql code whichmerges 2 table and finds the maximum value of salary column

[ASSISTANT]
{
  "id": "chatcmpl-7nZOFltiW5ysQmd9IMSUEDlaVdJmQ",
  "object": "chat.completion",
  "created": 1692048467,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "To perform this task, we need to know the names of the two tables and the structure of these tables, especially if the 'salary' column exists in both tables. However, I can provide a generic SQL query for your request:\n\n```sql\nSELECT MAX(salary) as MaxSalary\nFROM (\n  SELECT salary FROM table1\n  UNION ALL\n  SELECT salary FROM table2\n) as mergedTable\n```\n\nPlease replace 'table1' and 'table2' with your actual table names. This query will merge the 'salary' columns from both tables into one and find the maximum value."
      }
    }
  ],
